In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv('../data/movies.csv')
ratings = pd.read_csv('../data/ratings.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
len(ratings), len(movies)

(100836, 9742)

In [6]:
ratings.userId.nunique()

610

In [7]:
ml_matrix = ratings.pivot(index='movieId', columns='userId', values='rating').fillna(0)
ml_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
from scipy.sparse import csr_matrix

ml_csr = csr_matrix(ml_matrix.values)

In [9]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=17)
knn.fit(ml_csr[2:, :])

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=17)

In [10]:
distance, indices = knn.kneighbors(ml_csr[0], n_neighbors=20)

In [11]:
distance

array([[0.42739874, 0.4343632 , 0.43573831, 0.44261183, 0.45290409,
        0.45885465, 0.4589107 , 0.46108723, 0.46583124, 0.46961865,
        0.4720232 , 0.47214083, 0.47967548, 0.48196726, 0.48581831,
        0.48775438, 0.49068283, 0.49140738, 0.49145545, 0.49480359]])

In [12]:
indices

array([[2351,  416,  613,  222,  312,  320,  908,  544,  961,  966, 3187,
         504,  121,  255,  895,  813, 1180,   29,  275,   30]])

In [13]:
movie_df = movies.drop(index=[0, 1]).reset_index(drop=True)
movie_df.head()

,movieId,title,genres
0,3,Grumpier Old Men (1995),Comedy|Romance
1,4,Waiting to Exhale (1995),Comedy|Drama|Romance
2,5,Father of the Bride Part II (1995),Comedy
3,6,Heat (1995),Action|Crime|Thriller
4,7,Sabrina (1995),Comedy|Romance


In [14]:
for i in indices:
    print(movie_df['title'][i])

2351                                 'night Mother (1986)
416                                  Jurassic Park (1993)
613                  Independence Day (a.k.a. ID4) (1996)
222             Star Wars: Episode IV - A New Hope (1977)
312                                   Forrest Gump (1994)
320                                 Lion King, The (1994)
908     Once Upon a Time in the West (C'era una volta ...
544                            Mission: Impossible (1996)
961                                           Diva (1981)
966                           Arsenic and Old Lace (1944)
3187            Rififi (Du rififi chez les hommes) (1955)
504                                        Aladdin (1992)
121                                      Apollo 13 (1995)
255                                   Pulp Fiction (1994)
895                 Cheech and Chong's Up in Smoke (1978)
813            Willy Wonka & the Chocolate Factory (1971)
1180                                          Fall (1997)
29            

In [15]:
similarity, indices = knn.kneighbors(ml_csr[1], n_neighbors=20)
for i in indices:
    print(movie_df['title'][i])

320                                Lion King, The (1994)
434                                Mrs. Doubtfire (1993)
323                                     Mask, The (1994)
416                                 Jurassic Park (1993)
502                                    Home Alone (1990)
481               Nightmare Before Christmas, The (1993)
504                                       Aladdin (1992)
510                          Beauty and the Beast (1991)
16                 Ace Ventura: When Nature Calls (1995)
274                             Santa Clause, The (1994)
129                                        Casper (1995)
273                                      Stargate (1994)
174                                    Waterworld (1995)
335                                     True Lies (1994)
332                                         Speed (1994)
503                                         Ghost (1990)
215    Interview with the Vampire: The Vampire Chroni...
136                    Die Hard

In [16]:
pip install surprise

Note: you may need to restart the kernel to use updated packages.


In [17]:
from surprise import SVD, NMF
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection.split import train_test_split

In [18]:
data = Dataset.load_builtin('ml-100k')

trainset, testset = train_test_split(data, test_size=.2, random_state=42)

In [19]:
svd_algo = SVD(random_state = 42)
svd_algo.fit(trainset)

In [20]:
predictions_svd = svd_algo.test(testset)
predictions_svd[:10]

[Prediction(uid='907', iid='143', r_ui=5.0, est=4.611536250297749, details={'was_impossible': False}),
 Prediction(uid='371', iid='210', r_ui=4.0, est=4.37114223755087, details={'was_impossible': False}),
 Prediction(uid='218', iid='42', r_ui=4.0, est=3.738743910804353, details={'was_impossible': False}),
 Prediction(uid='829', iid='170', r_ui=4.0, est=3.8854605526859616, details={'was_impossible': False}),
 Prediction(uid='733', iid='277', r_ui=1.0, est=2.9512433241621805, details={'was_impossible': False}),
 Prediction(uid='363', iid='1512', r_ui=1.0, est=3.1175626582710105, details={'was_impossible': False}),
 Prediction(uid='193', iid='487', r_ui=5.0, est=3.584277708279981, details={'was_impossible': False}),
 Prediction(uid='808', iid='313', r_ui=5.0, est=4.694317679756497, details={'was_impossible': False}),
 Prediction(uid='557', iid='682', r_ui=2.0, est=3.188629840327263, details={'was_impossible': False}),
 Prediction(uid='774', iid='196', r_ui=3.0, est=2.4386385055444957, det

In [21]:
accuracy.rmse(predictions_svd)

RMSE: 0.9352


0.935171451026933

In [22]:
nmf_algo = NMF(random_state = 42)
nmf_algo.fit(trainset)

In [23]:
predictions_nmf = nmf_algo.test(testset)
predictions_nmf[:10]

[Prediction(uid='907', iid='143', r_ui=5.0, est=4.814679254593069, details={'was_impossible': False}),
 Prediction(uid='371', iid='210', r_ui=4.0, est=4.195125785827911, details={'was_impossible': False}),
 Prediction(uid='218', iid='42', r_ui=4.0, est=3.5255802741121975, details={'was_impossible': False}),
 Prediction(uid='829', iid='170', r_ui=4.0, est=4.131742088580094, details={'was_impossible': False}),
 Prediction(uid='733', iid='277', r_ui=1.0, est=3.709656181355704, details={'was_impossible': False}),
 Prediction(uid='363', iid='1512', r_ui=1.0, est=4.118810905573851, details={'was_impossible': False}),
 Prediction(uid='193', iid='487', r_ui=5.0, est=3.9782946152950114, details={'was_impossible': False}),
 Prediction(uid='808', iid='313', r_ui=5.0, est=4.942665763888022, details={'was_impossible': False}),
 Prediction(uid='557', iid='682', r_ui=2.0, est=2.6266692176433835, details={'was_impossible': False}),
 Prediction(uid='774', iid='196', r_ui=3.0, est=2.268692989867634, det

In [24]:
accuracy.rmse(predictions_nmf)

RMSE: 0.9594


0.9594077989819743

In [25]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [26]:
top_n = get_top_n(predictions_svd, n=10)
for uid, user_ratings in list(top_n.items())[:5]:
    print('user:', uid)
    print('movie recommendations: ', end='')
    for iid, _ in user_ratings:
        movie = movies.loc[movies['movieId'] == int(iid)]['title']
        if not movie.empty:
            print(movie.values[0], end=', ')
    print('\n')

user: 907
movie recommendations: Juror, The (1996), Johnny Mnemonic (1995), Courage Under Fire (1996), Flirting With Disaster (1996), Leaving Las Vegas (1995), Toy Story (1995), Net, The (1995), Judge Dredd (1995), Brothers McMullen, The (1995), 

user: 371
movie recommendations: Wild Bill (1995), Hate (Haine, La) (1995), Nine Months (1995), Forget Paris (1995), Kids (1995), Georgia (1995), Carlito's Way (1993), 

user: 218
movie recommendations: White Man's Burden (1995), Dracula: Dead and Loving It (1995), Dead Presidents (1995), Devil in a Blue Dress (1995), Striptease (1996), Seven (a.k.a. Se7en) (1995), True Crime (1996), 

user: 829
movie recommendations: 8 Seconds (1994), Safe (1995), Hackers (1995), Burnt by the Sun (Utomlyonnye solntsem) (1994), Strange Days (1995), From Dusk Till Dawn (1996), Anne Frank Remembered (1995), Girl 6 (1996), 

user: 733
movie recommendations: Nixon (1995), Toy Story (1995), Remains of the Day, The (1993), Sunset Blvd. (a.k.a. Sunset Boulevard) (19

In [27]:
top_n = get_top_n(predictions_nmf, n=10)
for uid, user_ratings in list(top_n.items())[:5]:
    print('user:', uid)
    print('movie recommendations: ', end='')
    for iid, _ in user_ratings:
        movie = movies.loc[movies['movieId'] == int(iid)]['title']
        if not movie.empty:
            print(movie.values[0], end=', ')
    print('\n')

user: 907
movie recommendations: Judge Dredd (1995), Larger Than Life (1996), Toy Story (1995), Juror, The (1996), Johnny Mnemonic (1995), Courage Under Fire (1996), Net, The (1995), Flirting With Disaster (1996), Santa Clause, The (1994), 

user: 371
movie recommendations: Kids (1995), Wild Bill (1995), Hate (Haine, La) (1995), Forget Paris (1995), Georgia (1995), Nine Months (1995), Carlito's Way (1993), 

user: 218
movie recommendations: Dracula: Dead and Loving It (1995), White Man's Burden (1995), Devil in a Blue Dress (1995), Dead Presidents (1995), Seven (a.k.a. Se7en) (1995), Striptease (1996), True Crime (1996), 

user: 829
movie recommendations: 8 Seconds (1994), Hackers (1995), Safe (1995), Pie in the Sky (1996), Strange Days (1995), Burnt by the Sun (Utomlyonnye solntsem) (1994), Girl 6 (1996), Circle of Friends (1995), Anne Frank Remembered (1995), 

user: 733
movie recommendations: Nixon (1995), Miracle on 34th Street (1994), Remains of the Day, The (1993), Toy Story (199

In [28]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


# New Test

In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
import pandas as pd

In [2]:
movies = pd.read_csv('../data/movies.csv')
ratings = pd.read_csv('../data/ratings.csv')

In [3]:
ratings = ratings.drop(columns='timestamp')
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [4]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings, reader)

In [5]:
trainset, testset = train_test_split(data, test_size=0.2)

# Train the SVD model
algo = SVD()
algo.fit(trainset)

In [13]:
# get name of movies 101, 103

movies[movies.title == 'Avengers, The (2012)']

,movieId,title,genres
7693,89745,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX
